In [1]:
import re
import datetime
import numpy as np
import pandas as pd 
import os
import sys

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.utils import *

### Establishes General Path

In [3]:
local_path = "/your/path/here"

In [4]:
path = '/Data/chicago-police-data/import'
db = dropbox_handler()
db.list_files(path+'/input')

['complaints-merged-2015_copy_20170112',
 'complaints-cpd-2016-dec_copy_20170112',
 'complaints-cpd-2016-jun_copy_20170112',
 'complaints-cpd-2016-nov_copy_20170112',
 'complaints-cpd-2016-oct_copy_20170112',
 'complaints-ipra-2016-apr_copy_20170112',
 'shootings-cpd-feb2016_copy_20170112',
 'shootings-ipra-may2016_copy_20170112',
 'TRRs']

In [19]:
in_path = path + '/input/complaints-cpd-2016-dec/'
out_path = local_path + '/output/complaints-cpd-2016-dec/'

In [20]:
location_code = db.download_file(path+'/doc/','Location_Code_Dictionary.csv')
location_code.dropna(how='all', inplace=True)
location_code['Location_Code']=location_code['Location_Code'].astype(int).astype(str)

def padding(value):
    if len(value)<2:
        return "0"+value
    else:
        return value
    
location_code['Location_Code'] = location_code['Location_Code'].apply(padding)

/Data/chicago-police-data/import/doc/location_code_dictionary.csv


## Dec 2016 Data

### Report 1

In [21]:
in_path_dec_2016_report = in_path
out_path_dec_2016_report = out_path

files = db.list_files(in_path_dec_2016_report)
files

['MAINFRAME_COMPLAINTS_REVISED_30NOV2016.CSV',
 'NEW WITNESS FILE NOV 29 2016 - no emp number.csv']

In [22]:
saving_files = [file.replace(" ","_").replace(".csv","").replace(".CSV","") for file in files]
saving_files

['MAINFRAME_COMPLAINTS_REVISED_30NOV2016',
 'NEW_WITNESS_FILE_NOV_29_2016_-_no_emp_number']

IS Investigation Unit Equivalent to Investigator_Assignment?

In [13]:
file=files[1]
final_df = pd.DataFrame()
metadata_df = pd.DataFrame()
##df = pd.read_csv(in_path_dec_2016_report + file)
df = db.download_file(in_path_dec_2016_report,file)

df.columns = ["CRID","Incident_Date","Incident_Time","Complaint_Date",
                  "Case_Status","Closed_Date","Investigator_Full_Name", 
                  "Investigator_Assignment","Beat",
               "Address_of_Incident", "Accused","Star","Accused_Gender",
              "Accused_Birth_Year","Accused_Appointed_Date","Current_Unit","Final_Complaint_Category"
              ,"Current_Rank","Complaint_Description","Final_Finding","Final_Finding_Description"
             , "Final_Action_Taken","Final_Action_Description"]

final_df = final_df.append(df)
final_df.reset_index(drop=True,inplace=True)
metadata_df = metadata_df.append(metadata_dataset(df,file))
metadata_df.reset_index(drop=True,inplace=True)

/Data/chicago-police-data/import/input/complaints-cpd-2016-dec_copy_20170112/mainframe_complaints_revised_30nov2016.csv


/Users/thudson/anaconda/envs/civis-model/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (2,8,11,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [16]:
df.columns.tolist()

['CRID',
 'Incident_Date',
 'Incident_Time',
 'Complaint_Date',
 'Case_Status',
 'Closed_Date',
 'Investigator_Full_Name',
 'Investigator_Assignment',
 'Beat',
 'Address_of_Incident',
 'Accused',
 'Star',
 'Accused_Gender',
 'Accused_Birth_Year',
 'Accused_Appointed_Date',
 'Current_Unit',
 'Final_Complaint_Category',
 'Current_Rank',
 'Complaint_Description',
 'Final_Finding',
 'Final_Finding_Description',
 'Final_Action_Taken',
 'Final_Action_Description']

In [ ]:
final_df.to_csv(out_path_dec_2016_report+saving_files[0]+".csv",index=False)
final_df.to_excel(out_path_dec_2016_report+saving_files[0]+".xlsx",index=False)

metadata_df.to_csv(out_path_dec_2016_report+saving_files[0]+"_metadata.csv",index=False)

### Report 2

In [17]:
file=files[0]
final_df = pd.DataFrame()
metadata_df = pd.DataFrame()
##df = pd.read_csv(in_path_dec_2016_report + file)
df = db.download_file(in_path_dec_2016_report,file)
print(df.head())
df.columns = ["CRID","Officer_Witness_or_Witness","Officer_Last_Name","Officer_First_Name",
                  "Star","Officer_Gender","Officer_Birth_Year","Officer_Race"]

final_df = final_df.append(df)
final_df.reset_index(drop=True,inplace=True)
metadata_df = metadata_df.append(metadata_dataset(df,file))
metadata_df.reset_index(drop=True,inplace=True)

/Data/chicago-police-data/import/input/complaints-cpd-2016-dec_copy_20170112/new witness file nov 29 2016 - no emp number.csv
  CASE NUMBER OFFICER OR NON OFFICER OFFICER LAST NAME OFFICER FIRST NAME  \
0     C052094    COMPLAINING_WITNESS               NaN                NaN   
1     C093392        OFFICER_WITNESS         BURZINSKI             WALTER   
2     C093392        OFFICER_WITNESS            SHAFER            CHARLES   
3     C100256    COMPLAINING_WITNESS               NaN                NaN   
4     C100256    COMPLAINING_WITNESS               NaN                NaN   

   OFFICER STAR NUMBER  SEX YEAR OF BIRTH RACE  
0                  NaN    M          1988    X  
1              13037.0  NaN           NaN  NaN  
2               7288.0  NaN           NaN  NaN  
3                  NaN    F          1946    S  
4                  NaN    M          1958    S  


In [18]:
df.columns.tolist()

['CRID',
 'Officer_Witness_or_Witness',
 'Officer_Last_Name',
 'Officer_First_Name',
 'Star',
 'Officer_Gender',
 'Officer_Birth_Year',
 'Officer_Race']

In [ ]:
final_df.to_csv(out_path_dec_2016_report+saving_files[1]+".csv",index=False)
final_df.to_excel(out_path_dec_2016_report+saving_files[1]+".xlsx",index=False)

metadata_df.to_csv(out_path_dec_2016_report+saving_files[1]+"_metadata.csv",index=False)